In [ ]:
# begin_block = 5905762
# end_block = begin_block + 3600/15*24*30*12

# tasks = (begin_block..end_block).each_slice(100000).to_a.map {|x| {status:"open",range:x} }

# puts "begin_block = #{begin_block}"
# puts "end_block = #{end_block}"
# puts tasks.size

# # Create a simple file store
# store = Moneta.new(:File, dir: '/home/jovyan/work/loan_data')

# # Store some entries
# store['fomo_tasks'] = tasks

# # # Read entry
# # store.key?('loan_data') # returns true
# # read_data = store['loan_data'] # returns 'value'

# store.close

begin_block = 5905762
end_block = 7979362
21


In [ ]:
# npx hardhat node --fork https://eth-mainnet.alchemyapi.io/v2/xOu9KmQYmgmqBuYhhPW0naOB9YRY3foa
# ngrok authtoken 1WA2UnsI5GHgexqADDhvw7v1Npd_dqSFiM3bX98ny1vjvx3a
# ngrok http 8545

# 初始化
require 'ethereum.rb'
require 'eth'
require 'faraday'
require 'parallel'
require 'resolv-replace'
require 'erb'    
require 'moneta'

# $client = Ethereum::HttpClient.new("http://127.0.0.1:8545/")
$client = Ethereum::HttpClient.new("https://eth-mainnet.alchemyapi.io/v2/xOu9KmQYmgmqBuYhhPW0naOB9YRY3foa")



#<Ethereum::HttpClient:0x00007ff1b414f228 @id=0, @log=false, @batch=nil, @formatter=#<Ethereum::Formatter:0x00007ff1b414f200>, @gas_price=22000000000, @gas_limit=4000000, @host="eth-mainnet.alchemyapi.io", @port=443, @proxy=nil, @ssl=true, @uri=#<URI::HTTPS https://eth-mainnet.alchemyapi.io/v2/xOu9KmQYmgmqBuYhhPW0naOB9YRY3foa>>

In [ ]:
# Create a simple file store
store = Moneta.new(:File, dir: '/home/jovyan/work/loan_data')

# # Read entry
store.key?('fomo_tasks') # returns true
tasks = store['fomo_tasks'] # returns 'value'

task_range = nil
tasks = tasks.map {|x| 
  if x[:status]=="open" and task_range==nil then
    x[:status]="run"
    task_range = x[:range]
    x
  else
    x
  end
}

# Store some entries
store['fomo_tasks'] = tasks
store.close

if task_range!=nil then
    puts task_range.count
    puts task_range[0,10]
end

100000
5905762
5905763
5905764
5905765
5905766
5905767
5905768
5905769
5905770
5905771


In [ ]:
def auto_retry(logger=nil,retry_cnt=12)
    begin
        yield
    rescue IOError,Faraday::TimeoutError,Net::OpenTimeout,OpenSSL::SSL::SSLError,JSON::ParserError,Net::ReadTimeout,Errno::EPIPE,Errno::ECONNRESET,Errno::ECONNREFUSED=>e
        if (retry_cnt-=1) > 0 then
            retry_number = 12-retry_cnt
            
            logger.call "sleep #{0.01*(2**retry_number)}" if logger!=nil
            sleep (0.01*(2**retry_number))
            logger.call "retry_cnt #{retry_number} at #{Time.now.to_s}" if logger!=nil
            retry 
        else
            raise e
        end
    end
end

# auto_retry(lambda {|x| puts(x.to_s)},12) { raise Net::OpenTimeout }
method_map = {"0x8f38f309"=>true,"0x98a0871d"=>true,"0xa65b37a1"=>true}


{"0x8f38f309"=>true, "0x98a0871d"=>true, "0xa65b37a1"=>true}

In [ ]:
time = Time.now

fomo_contracts = Parallel.map(task_range, in_threads: 30) { |cur_block|
    puts cur_block if cur_block % 10000 == 0
    tx = auto_retry(lambda {|x| puts(x.to_s)},12) { $client.eth_get_block_by_number(cur_block,true)["result"]["transactions"] }
    tx.filter {|x| method_map[x["input"][0,10]]}.map { |x| x["to"] }.uniq
}.flatten.uniq

puts "time - #{Time.now-time}s"


# Create a simple file store
store = Moneta.new(:File, dir: '/home/jovyan/work/loan_data')

# # Read entry
store.key?('fomo_all') # returns true
fomo_all = store['fomo_all'] # returns 'value'

puts "fomo_all.count - #{fomo_all.count}"
fomo_all = (fomo_all + fomo_contracts).uniq
puts "fomo_contracts.count - #{fomo_contracts.count}"
puts "fomo_all.count - #{fomo_all.count}"

# Store some entries
store['fomo_all'] = fomo_all
store.close


5910000
5920000
5930000
sleep 0.02


In [ ]:
# load init moudle file

load "/work/pannote/panbot_note.rb"
nil

In [ ]:
contracts = Task.where(name:"misc/fomo_contract").where(status:"close").map {|x| x.raw_ret}.flatten.uniq

["0xdd9fd6b6f8f7ea932997992bbe67eabb3e316f3c", "0xa004b902bdcf710462f905ed487642031d17521c", "0x4e8ecf79ade5e2c49b9e30d795517a81e0bf00b8", "0x4f0b09be4032772ff023ab374b45c615ffd3f4e5", "0xa62142888aba8370742be823c1782d17a0389da1", "0xc65d85f7bda97015c0fe23b8cdca3c5567a526a1", "0x9872ffc47ff6ae0cbdec2f68bb88ad3169d69afc", "0x820dfa17d30f938dc2c172b716630a06ec759d99", "0x6db943251e4126f913e9733821031791e75df713", "0xe336d0c8b24d8e20b6f714522f590abadb601493", "0x0bacc180c94648daffef39c4ab7a9e5e6657230e", "0xcfcd4a9607d756704ac52d28777f98a48ee99efb", "0xed3762edcc5820a5587105d7b9f574c711c4a700", "0x3d9806b5f595398b27706729706c86096dfa4197", "0x3dd21350a340adfc0a040e0c7074ec8f5728062a", "0x81675fc9d2395b1fbc606cf97d7c488d899d201c", "0x0ad3227eb47597b566ec138b3afd78cfea752de5", "0x4d2c5308f1d9b42bdc398d215713bbc9e0738a0a", "0x86d179c28cceb120cd3f64930cf1820a88b77d60", "0xc2748e342411a2b1f84f9fa807d4e1c81365da1c", "0x342fb83e089e3e04ba4dc4e21c21ec44a9ec0c8e", "0x103d98686ced96f1d2cf1a0d1eabdd

In [ ]:
contracts.count

483

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>